In [19]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import math
import datetime as dt

In [20]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["figure.figsize"] = (7,5)

In [21]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

## Funcs

In [22]:
## 극단값 보정 # 윈저라이징 #3std
def clean_outlier(srs, n = 3):
    srs = srs.copy()
    ut = srs.mean() + n*srs.std()
    lt = srs.mean() - n*srs.std()
    srs[(srs > ut)] = ut
    srs[(srs < lt)] = lt
    return srs

## Data

In [23]:
macro = pd.read_csv('data/macro_encoded_sav.csv', index_col=0)
macro.index = pd.to_datetime(macro.index).dropna()

assets = pd.read_csv('data/assets_encoded_sav.csv', index_col=0)
assets.index = pd.to_datetime(assets.index).dropna()

macro_ori = pd.read_csv('data/macro.csv', index_col=0).pct_change().dropna()
macro_ori.index = pd.to_datetime(macro_ori.index).dropna()
assets_ori = pd.read_csv('data/assets.csv', index_col=0).pct_change().dropna()
assets_ori.index = pd.to_datetime(assets_ori.index).dropna()

In [24]:
## 구글트랜드
GT_ori = pd.read_csv('data/GT_volume_20220411.csv', index_col=0).pct_change().dropna()
GT_ori.index = pd.to_datetime(GT_ori.index)
GT_ori = GT_ori.resample('M').last()

In [25]:
asss = ['SPY', 'QQQ', 'VEA', 'VWO', 'TLT', 'IEF', 'SHY', 'IAU']
assets = assets[asss]

### 변수선택

In [26]:
## 분석 대상 데이터 ## 선행성
dataset = macro.dropna().shift(1).dropna()
## 이상값 제거
dataset_v2 = dataset.dropna()
for mac in dataset_v2.columns:
    dataset_v2[mac] = clean_outlier(dataset_v2[mac], n = 2)

## 활용설명변수
dataset_v3 = dataset_v2.dropna()
X_ = dataset_v3[['VIX', 'DG10', 'DG2', 'SPPE']]
## 수익률
df_rets = assets.dropna()
## 자산별 12-1M
df121M = assets.rolling(12).sum().shift(1).dropna()
df121M.columns = df121M.columns + '12-1M'


## 타겟자산과 다른자산(-1) 상관계수
df_ls = []
for col in df_rets.columns:
    assets_ = list(df_rets.columns)
    assets_.remove(col)
    temp = df_rets[assets_].shift(1)
    temp[col] = df_rets[col]
    temp_corr = temp.corr()[col]
    df_ls.append(temp)

## 후보 변수 결합
df_ls2 = []
for i in range(len(df_ls)):
    temp = pd.concat([X_, df121M, df_ls[i]], axis=1).dropna()
    df_ls2.append(temp)
    
## 상관계수 0.1이상 변수만 남김
df_ls3 = []
for i in range(len(df_ls2)):
    df = df_ls2[i]
    temp_corr = df.corr()[df.columns[-1]] 
    df_ls3.append(df[temp_corr[abs(temp_corr) >= .25].index])
    
## vif 10이하 변수만 남김
df_ls4 = []
for i in range(len(df_ls3)):
    df = df_ls3[i]
    X = df[df.columns[:-1]] 
    vif = pd.DataFrame()
    vif["features"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    temp = df[vif['features'][vif['VIF'] < 10]]
    temp[df.columns[-1]] = df[df.columns[-1]]
    df_ls4.append(temp)

### Set 구성

In [27]:
## 모델링 대상 데이터 lag없음
dataset = macro.dropna().dropna()
## 이상값 제거
dataset_v2 = dataset.dropna()
for mac in dataset_v2.columns:
    dataset_v2[mac] = clean_outlier(dataset_v2[mac], n = 2)

## 활용설명변수
dataset_v3 = dataset_v2.dropna()
X_ = dataset_v3[['VIX', 'DG10', 'DG2', 'SPPE']]
## 수익률
df_rets = assets.dropna()
## 자산별 12-1M
df121M = assets.rolling(12).sum().shift(1).dropna()
df121M.columns = df121M.columns + '12-1M'

df_ = []
for col in df_rets.columns:
    assets_ = list(df_rets.columns)
    assets_.remove(col)
    temp = df_rets[assets_]
    temp[col] = df_rets[col]
    df_.append(temp)

## 후보 변수 결합
dfs = []
for i in range(len(df_ls4)):
    temp = pd.concat([X_, df121M, df_[i]], axis=1).dropna()
    temp = temp[df_ls4[i].columns]
    dfs.append(temp)

In [28]:
df_SPY, df_QQQ, df_VEA, df_VWO, df_TLT, df_IEF, df_SHY, df_IAU = dfs

In [121]:
df_SPY.to_csv('data/engineered/edf_SPY.csv')
df_QQQ.to_csv('data/engineered/edf_QQQ.csv')
df_VEA.to_csv('data/engineered/edf_VEA.csv')
df_VWO.to_csv('data/engineered/edf_VWO.csv')
df_TLT.to_csv('data/engineered/edf_TLT.csv')
df_IEF.to_csv('data/engineered/edf_IEF.csv')
df_SHY.to_csv('data/engineered/edf_SHY.csv')
df_IAU.to_csv('data/engineered/edf_IAU.csv')
## df_XLF.to_csv('data/edf_XLF.csv')

In [29]:
## 필터 없는 원천데이터에서 변수선택만한 버전(인코더용)

## 수익률
ori_rets = assets_ori.dropna()
## 자산별 12-1M
ori121M = assets_ori.rolling(12).sum().shift(1).dropna()
ori121M.columns = ori121M.columns + '12-1M'

temp = pd.concat([ori121M, X_, ori_rets], axis=1).dropna()

df_SPY_ori = temp[df_SPY.columns]
df_QQQ_ori = temp[df_QQQ.columns]
df_VEA_ori = temp[df_VEA.columns]
df_VWO_ori = temp[df_VWO.columns]
df_TLT_ori = temp[df_TLT.columns]
df_IEF_ori = temp[df_IEF.columns]
df_SHY_ori = temp[df_SHY.columns]
df_IAU_ori = temp[df_IAU.columns]

In [36]:
## 구글트랜드 병합
df_SPY_gt =  pd.concat([df_SPY_ori, GT_ori], axis=1).dropna()
df_QQQ_gt =  pd.concat([df_QQQ_ori, GT_ori], axis=1).dropna()
df_VEA_gt =  pd.concat([df_VEA_ori, GT_ori], axis=1).dropna()
df_VWO_gt =  pd.concat([df_VWO_ori, GT_ori], axis=1).dropna()
df_TLT_gt =  pd.concat([df_TLT_ori, GT_ori], axis=1).dropna()
df_IEF_gt =  pd.concat([df_IEF_ori, GT_ori], axis=1).dropna()
df_SHY_gt =  pd.concat([df_SHY_ori, GT_ori], axis=1).dropna()
df_IAU_gt =  pd.concat([df_IAU_ori, GT_ori], axis=1).dropna()

In [135]:
## 변수선택 없음
df_SPY_ori = temp[['SPY12-1M', 'QQQ12-1M', 'VEA12-1M', 'VWO12-1M', 'TLT12-1M', 'IEF12-1M',
                   'SHY12-1M', 'IAU12-1M', 'XLF12-1M', 'VIX', 'DG10', 'DG2', 'SPPE',
                   'QQQ', 'VEA', 'VWO', 'TLT', 'IEF', 'SHY', 'IAU', 'XLF', 'SPY']]
df_QQQ_ori = temp[['SPY12-1M', 'QQQ12-1M', 'VEA12-1M', 'VWO12-1M', 'TLT12-1M', 'IEF12-1M',
                   'SHY12-1M', 'IAU12-1M', 'XLF12-1M', 'VIX', 'DG10', 'DG2', 'SPPE', 'SPY',
                   'VEA', 'VWO', 'TLT', 'IEF', 'SHY', 'IAU', 'XLF', 'QQQ']]
df_VEA_ori = temp[['SPY12-1M', 'QQQ12-1M', 'VEA12-1M', 'VWO12-1M', 'TLT12-1M', 'IEF12-1M',
                   'SHY12-1M', 'IAU12-1M', 'XLF12-1M', 'VIX', 'DG10', 'DG2', 'SPPE', 'SPY',
                   'QQQ', 'VWO', 'TLT', 'IEF', 'SHY', 'IAU', 'XLF', 'VEA']]
df_VWO_ori = temp[['SPY12-1M', 'QQQ12-1M', 'VEA12-1M', 'VWO12-1M', 'TLT12-1M', 'IEF12-1M',
                   'SHY12-1M', 'IAU12-1M', 'XLF12-1M', 'VIX', 'DG10', 'DG2', 'SPPE', 'SPY',
                   'QQQ', 'VEA', 'TLT', 'IEF', 'SHY', 'IAU', 'XLF', 'VWO']]
df_TLT_ori = temp[['SPY12-1M', 'QQQ12-1M', 'VEA12-1M', 'VWO12-1M', 'TLT12-1M', 'IEF12-1M',
                   'SHY12-1M', 'IAU12-1M', 'XLF12-1M', 'VIX', 'DG10', 'DG2', 'SPPE', 'SPY',
                   'QQQ', 'VEA', 'VWO', 'IEF', 'SHY', 'IAU', 'XLF', 'TLT']]
df_IEF_ori = temp[['SPY12-1M', 'QQQ12-1M', 'VEA12-1M', 'VWO12-1M', 'TLT12-1M', 'IEF12-1M',
                   'SHY12-1M', 'IAU12-1M', 'XLF12-1M', 'VIX', 'DG10', 'DG2', 'SPPE', 'SPY',
                   'QQQ', 'VEA', 'VWO', 'TLT', 'SHY', 'IAU', 'XLF', 'IEF']]
df_SHY_ori = temp[['SPY12-1M', 'QQQ12-1M', 'VEA12-1M', 'VWO12-1M', 'TLT12-1M', 'IEF12-1M',
                   'SHY12-1M', 'IAU12-1M', 'XLF12-1M', 'VIX', 'DG10', 'DG2', 'SPPE', 'SPY',
                   'QQQ', 'VEA', 'VWO', 'TLT', 'IEF', 'IAU', 'XLF', 'SHY']]
df_IAU_ori = temp[['SPY12-1M', 'QQQ12-1M', 'VEA12-1M', 'VWO12-1M', 'TLT12-1M', 'IEF12-1M',
                   'SHY12-1M', 'IAU12-1M', 'XLF12-1M', 'VIX', 'DG10', 'DG2', 'SPPE', 'SPY',
                   'QQQ', 'VEA', 'VWO', 'TLT', 'IEF', 'SHY', 'XLF', 'IAU']]

In [136]:
df_SPY_ori.to_csv('data/engineered/edf_SPY_ori.csv')
df_QQQ_ori.to_csv('data/engineered/edf_QQQ_ori.csv')
df_VEA_ori.to_csv('data/engineered/edf_VEA_ori.csv')
df_VWO_ori.to_csv('data/engineered/edf_VWO_ori.csv')
df_TLT_ori.to_csv('data/engineered/edf_TLT_ori.csv')
df_IEF_ori.to_csv('data/engineered/edf_IEF_ori.csv')
df_SHY_ori.to_csv('data/engineered/edf_SHY_ori.csv')
df_IAU_ori.to_csv('data/engineered/edf_IAU_ori.csv')

In [37]:
## 구글트랜드 추가
df_SPY_gt.to_csv('data/engineered/df_SPY_gt.csv')
df_QQQ_gt.to_csv('data/engineered/df_QQQ_gt.csv')
df_VEA_gt.to_csv('data/engineered/df_VEA_gt.csv')
df_VWO_gt.to_csv('data/engineered/df_VWO_gt.csv')
df_TLT_gt.to_csv('data/engineered/df_TLT_gt.csv')
df_IEF_gt.to_csv('data/engineered/df_IEF_gt.csv')
df_SHY_gt.to_csv('data/engineered/df_SHY_gt.csv')
df_IAU_gt.to_csv('data/engineered/df_IAU_gt.csv')